In [ ]:
from pathlib import Path
import re
import pandas as pd

# Read the rules
txt_path = Path("rules_student_pretty.txt")
raw_txt = txt_path.read_text(encoding="utf-8")

# Extract (rule, class)
pattern = r"IF\s*\((.*?)\)\s*THEN\s*(\w+)"
pairs = re.findall(pattern, raw_txt, flags=re.DOTALL)

# Escape LaTeX
def escape_latex(text: str) -> str:
    rep = {
        '&': r'\&', '%': r'\%', '$': r'\$', '#': r'\#', '_': r'\_',
        '{': r'\{', '}': r'\}', '~': r'\textasciitilde{}',
        '^': r'\textasciicircum{}', '\\': r'\textbackslash{}'
    }
    text = ''.join(rep.get(c, c) for c in text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Build LaTeX table with horizontal separators when the class changes
header = r"""\begin{longtable}{p{12cm}l}
\toprule
\textbf{Regola} & \textbf{Classe}\\
\midrule
\endhead
"""

rows = []
last_class = None
i = 0
for rule, cls in pairs:
    if i < 15:
        rule_esc = escape_latex(rule)
        cls_esc = escape_latex(cls)
        # Add separator if class changes
        if last_class is not None and cls_esc != last_class:
            rows.append(r"\midrule")
        rows.append(f"{rule_esc} & {cls_esc} \\\\")
        last_class = cls_esc
    i += 1

footer = r"""\bottomrule
\end{longtable}
"""

latex_table = header + "\n".join(rows) + "\n" + footer

# Save result
output_path = Path("rules_table_separators.txt")
output_path.write_text(latex_table, encoding="utf-8")

# Show preview
print("\n".join(latex_table.splitlines()[:25]))
